In [1]:
import os
import pickle
# import pandas as pd
from utils_mocap_viz.generate_views import (    # organize this
    get_output_dir,
    prepare_videos
)
from utils_mocap_viz.animated_phase_analysis import generate_all_animations
from utils_mocap_viz.animated_merged_phase_analysis import animate_merged_phase_analysis
from utils_mocap_viz.kinematic_visualizer import visualize_joint_position
from utils_mocap_viz.video_layout import combine_views, get_video_paths

# from utils_subdivision.gen_distribution_single_plots import find_cycle_phases, kde_estimate

from utils_dance_anim.dance_dot import animate_dance_phase_analysis, save_dance_phase_plot

###  Configuration

In [ ]:
filename = "BKO_E1_D1_02_Maraka"
bvh_dir = os.path.join("data", "bvh_files")
bvh_file = os.path.join(bvh_dir, filename + "_T")

mode = ["group", "individual", "audience"]
m_idx = 0

# path to onsets and cycles csv files
cycles_csv_path = f"data/virtual_cycles/{filename}_C.csv"
onsets_csv_path = f"data/drum_onsets/{filename}.csv"
dmode_path = f"data/dance_modes_ts/{filename}_{mode[m_idx]}.pkl"

with open(dmode_path, "rb") as f:
    dmode = pickle.load(f)


print(f"Time segments for {mode[m_idx]}:",dmode)

In [ ]:
start_time, end_time = dmode[0]
print(start_time, end_time)

# prepare output directory
output_dir = get_output_dir(bvh_file, start_time, end_time)
output_fps = 24

### Generate Separate Drum distribution video plots

In [ ]:
generate_all_animations(
    filename, start_time, end_time,
    cycles_csv_path, onsets_csv_path,
    save_dir=output_dir,
    figsize=(10, 3), dpi=200
)

### Generate merged distribution video plot (DunDun, J1, J2)

In [ ]:
save_fname = "drum_combined.mp4"

animate_merged_phase_analysis(
    filename, start_time, end_time,
    cycles_csv_path, onsets_csv_path,
    figsize=(10, 3), dpi=200,
    save_fname = save_fname,
    save_dir=output_dir
    )

### Generate Skeleton Videos + trimmed video_mix + audio

In [ ]:
# bvh_file = os.path.join(bvh_dir, filename + "_T")
video_path = os.path.join("data", "videos", f"{filename}_pre_R_Mix.mp4")
video_size = (640, 360)

# views_to_generate = ['front', 'right']

output_dir, view_videos = prepare_videos(
    filename= bvh_file,
    start_time= start_time,
    end_time= end_time,
    video_path= video_path,
    video_size= video_size,
    fps= output_fps
)

### Generate animated kinematic plots

In [6]:
joint_name = "LeftAnkle"  
axis = 'y'      # y is vertical in bvh files

In [ ]:
# Generate joint position visualization

joint_video = os.path.join(output_dir, f"{joint_name}_{axis}_position.mp4")
if not os.path.exists(joint_video):
    visualize_joint_position(
        bvh_file=bvh_file + ".bvh",
        joint_name= joint_name,
        axis= axis,
        start_time= start_time,
        end_time= end_time,
        output_fps= output_fps,
        output_dir= output_dir,
        fig_size= (12, 4),  # Half height for joint visualization
        dpi= 200
    )

## 02 June: Define layouts + Combine Videos

In [ ]:
paths = get_video_paths(output_dir, filename, joint_name, axis)
paths

In [ ]:
# ────────────────────────────────────────────────────────────────────────────────
#  Define layouts (must reference keys returned by get_video_paths)
# ────────────────────────────────────────────────────────────────────────────────
layout = {
    'L1': [
        {'view': 'front',       'x': 0,   'y': 0,   'width': 320,  'height': 360},
        {'view': 'right',       'x': 320, 'y': 0,   'width': 320,  'height': 360},
        {'view': 'video_mix',   'x': 640, 'y': 0,   'width': 640,  'height': 360},
        {'view': 'combined_drum',    'x': 0,   'y': 360, 'width': 1280, 'height': 360}
    ],
    'L2': [
        {'view': 'front',         'x': 0,   'y': 0,   'width': 640,  'height': 360},
        {'view': 'right',         'x': 640, 'y': 0,   'width': 640,  'height': 360},
        {'view': 'joint_pos',     'x': 0,   'y': 360, 'width': 1280, 'height': 360}
    ]
}

canvas_size = (1280, 720)
# joint_name = "LeftAnkle"
# axis = "y"

# Choose which layout to use:
layout_config = layout['L1']

# ────────────────────────────────────────────────────────────────────────────────
# 2.  Obtain video paths for all possible views
#     (get_video_paths returns keys like 'front', 'right', 'combined',
#      'video_mix', 'joint_pos', etc.)
# ────────────────────────────────────────────────────────────────────────────────
view_videos = get_video_paths(
    output_dir=output_dir,
    filename=filename,
    joint_name=joint_name,
    axis=axis
)

# ────────────────────────────────────────────────────────────────────────────────
# Check for any missing videos required by the chosen layout
# ────────────────────────────────────────────────────────────────────────────────
required_views = {item['view'] for item in layout_config}
missing_views = required_views - set(view_videos.keys())

if missing_views:
    print(f"Warning: Missing videos for views: {missing_views}")
    print("Please generate those videos before attempting to combine.")


# ────────────────────────────────────────────────────────────────────────────────
# 4.  Combine the videos using the specified layout
# ────────────────────────────────────────────────────────────────────────────────
combine_views(
    filename= bvh_file, 
    start_time= start_time, 
    end_time= end_time,
    output_dir= output_dir, 
    view_videos= view_videos,
    layout_config= layout_config, 
    video_size= canvas_size, 
    fps= output_fps
)


### Generate Foot Onset Distribution Video Plot

In [ ]:
dance_csv_path = f"data/dance_onsets/{filename}_T_dance_onsets.csv"
animate_dance_phase_analysis(
    filename, start_time, end_time,
    cycles_csv_path, dance_csv_path,
    figsize= (10, 3), dpi= 200, save_dir= output_dir
)

## Raw trajectory animation

In [ ]:
import os
import numpy as np
import matplotlib
# matplotlib.use('Agg')  # Use non-interactive backend
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import pandas as pd

def animate_trajectories(
    file_name: str,
    W_start: float,
    W_end: float,
    base_path_logs: str = "data/logs_v1_may",
    frame_rate: float = 240,
    figsize: tuple = (10, 3),
    dpi: int = 100,
    save_dir: str = None
):
    """
    Animate the raw foot trajectories with a moving vertical playhead.
    """
    # Build file paths
    logs_onset_dir = os.path.join(base_path_logs, f"{file_name}_T", "onset_info")
    left_zpos_csv = os.path.join(logs_onset_dir, f"{file_name}_T_left_foot_zpos.csv")
    right_zpos_csv = os.path.join(logs_onset_dir, f"{file_name}_T_right_foot_zpos.csv")

    # Load data
    Lz = pd.read_csv(left_zpos_csv)["zpos"].values
    Rz = pd.read_csv(right_zpos_csv)["zpos"].values
    n_frames = len(Lz)
    times = np.arange(n_frames) / frame_rate

    # Trim to window
    win_mask = (times >= W_start) & (times <= W_end)
    t_win = times[win_mask]
    L_win = Lz[win_mask]
    R_win = Rz[win_mask]

    # Create figure and axis
    fig, ax = plt.subplots(figsize=figsize, dpi=dpi)
    
    # Plot trajectories
    ax.plot(t_win, L_win, '-', color='blue', alpha=0.5, label='Left Foot')
    ax.plot(t_win, R_win, '--', color='red', alpha=0.5, label='Right Foot')
    
    # Set y-axis limits based on the data
    y_min = min(L_win.min(), R_win.min())
    y_max = max(L_win.max(), R_win.max())
    y_range = y_max - y_min
    ax.set_ylim(y_min - 0.1*y_range, y_max + 0.1*y_range)
    
    # Create vertical playhead
    v_playhead, = ax.plot([W_start, W_start], [y_min - 0.1*y_range, y_max + 0.1*y_range], 
                         'k-', lw=2, alpha=0.7)
    
    # Set up the plot
    ax.set_xlabel('Time (s)')
    ax.set_ylabel('Foot Position')
    ax.set_title(f'File: {file_name} | Window: {W_start:.1f}s - {W_end:.1f}s')
    ax.grid(True, alpha=0.3)
    ax.legend(loc='upper left')
    
    def update(frame):
        """Update function for animation."""
        # Update vertical playhead position
        v_playhead.set_xdata([frame, frame])
        # Update title with current time
        ax.set_title(f'File: {file_name} | Window: {W_start:.1f}s - {W_end:.1f}s | Time: {frame:.2f}s')
        return v_playhead,
    
    # Create animation
    print("\nCreating animation...")
    frames = np.arange(W_start, W_end, 0.05)  # 50ms steps
    print(f"Animation will have {len(frames)} frames")
    print(f"Time range: {frames[0]:.2f}s - {frames[-1]:.2f}s")
    
    anim = animation.FuncAnimation(
        fig, update, frames=frames,
        interval=50, blit=True
    )
    
    # Apply tight_layout before saving
    plt.tight_layout()
    
    if save_dir:
        # Create save directory if it doesn't exist
        os.makedirs(save_dir, exist_ok=True)
        
        # Create filename
        save_filename = f"{file_name}_trajectories.mp4"
        save_path = os.path.join(save_dir, save_filename)
        
        print(f"\nSaving animation to: {save_path}")
        try:
            # Save animation as MP4
            writer = animation.FFMpegWriter(fps=24, bitrate=2000)
            anim.save(save_path, writer=writer)
            plt.close(fig)  # Explicitly close the figure
            print("Animation saved successfully!")
        except Exception as e:
            print(f"Error saving animation: {str(e)}")
            plt.close(fig)  # Close figure even if there's an error
    else:
        print("Error: save_dir must be provided")
        plt.close(fig)  # Close figure if no save directory
    
    return anim

if __name__ == "__main__":
    # Example usage
    file_name = "BKO_E1_D2_03_Suku"
    W_start = 60.0
    W_end = 80.0
    figsize = (10, 3)
    dpi = 150
    save_dir = "trajectory_animations"
    
    animate_trajectories(
        file_name, W_start, W_end,
        figsize=figsize, dpi=dpi,
        save_dir=save_dir
    )